In [ ]:
import dask.dataframe as dd
from dask.distributed import Client
import json
import dask
import numpy as np
import re
# import graphviz
import pandas as pd
import random

In [ ]:
client=Client('172.31.42.114:8786')
client
# client.scheduler_info()
# #dask.config.config

In [ ]:
products = dd.read_csv("small_products_Release.csv")
reviews = dd.read_csv("small_user_reviews_Release.csv")

In [ ]:
products.head(4)

In [ ]:
reviews.head(1)

In [ ]:
products.shape[0].compute()

In [ ]:
products.head()


In [ ]:
reviews.isna().mean().compute()

In [ ]:
products.isna().mean().compute()

In [ ]:
subproducts = products[['asin', 'price']]
subreviews = reviews[['asin', 'overall']]

#optimise with left and not index on products
joined = subproducts.join(subreviews.set_index("asin"), on="asin", how='inner')
#joined.head()
pcor = joined.corr()['overall']
val = pcor.compute()['price']
val

In [ ]:
price_prod = products['price'].describe().compute()
#check if min is actuatly returning empty
# [price_prod['mean'], price_prod['std'], price_prod['50%'], price_prod['min'], price_prod['max']]
price_prod[['mean', 'std', '50%', 'min', 'max']]

In [ ]:
def f(x):
    if isinstance(x, float):
        return x
    else:
        return x.strip("[]").split(',')[0]

numCat = products.assign(categories = lambda x: x.categories.apply(
    f,
    meta=('categories', 'str'))
                        )

numCat = numCat.groupby("categories").count()['asin']
numCat.compute()[1:]

#[1:]this removes the nan catagory

In [ ]:
# #Q5

# subR = reviews[['asin']].drop_duplicates().set_index('asin')

# subP = products.set_index('asin')

# iner = subR.join(subP, on="asin", how='inner').shape[0]

# left = subR.join(subP, on="asin", how='left').shape[0]

# # print(iner)
# # print(left)

# # out = (iner == left).compute()

# # out
# iner.head()
# #products.set_index('asin').index
# #index.difference(products.set_index('asin').index)

In [ ]:
rev_data = ['A' + str(i) for i in range(170)]
prod_data = ['A' + str(i) for i in range(160)]
rev_data.extend(['A' + str(i) for i in range(20)])


test_reviews = dd.from_pandas(
    pd.DataFrame(
        {'asin':rev_data}
    ),
    npartitions = 16
)
test_products = dd.from_pandas(
    pd.DataFrame(
        {'asin':prod_data}
    ),
    npartitions = 16
)
test_reviews.tail(10)
# test_products.head(10)


In [ ]:
# # Q5 (without index)
subR = reviews[['asin']]

subP = products[["asin"]]
subP["temp"] = 1

left = dd.merge(subR, subP, how = "left")

left.temp.isna().any().compute() + 0

In [ ]:
from string import ascii_uppercase
from string import digits
random.seed(100)

In [ ]:
def rand_asin():
    return ''.join(random.choice(ascii_uppercase + digits) for i in range(10))
def rand_related(X, n):
    return str(random.sample(X, k = random.randint(1,n)))

prod_data = [rand_asin() for i in range(160)]
rev_data = [rand_related(prod_data, 10) for i in range(160)]

test_reviews = dd.from_pandas(
    pd.DataFrame(
        {'related':rev_data}
    ),
    npartitions = 16
)
test_products = dd.from_pandas(
    pd.DataFrame(
        {'asin':prod_data}
    ),
    npartitions = 16
)

test_reviews.head(10)

In [ ]:
X_test = dd.from_pandas(
    pd.DataFrame(
        {'x':[rand_related(prod_data, 10) for i in range(5)]}
    ),
    npartitions = 2
)
X_test.apply(lambda x: x + "testingstr", axis=1, meta=('x', 'str')).compute()


In [ ]:
df = pd.DataFrame({'x': [1, 2, 3, 4, 5],
                   'y': [1., 2., 3., 4., 5.]})

def parse(related):
    return "|".join(re.findall("[A-Z0-9]{10}", related))

ddf = dd.from_pandas(df, npartitions=2)
test_reviews[["related"]].apply(
    parse, axis=1, meta=('related', 'str')
).compute()

In [ ]:
#Q6

subP = products

ser = subP["asin"]

def parse(related):
    return "|".join(re.findall("[A-Z0-9]{10}", related))

def f(val):
    asins = parse(val)
    return ser.str.contains("|".join(asins)).any()
    
def g(df):
    df['related'].apply(f,meta="str")

#f({"related":"0000143561"}).compute()
#f({"related":"B0036FO6SI"}).compute()

subP.assign(related = g,meta="str").compute()



In [ ]:
subP = products.set_index("asin")


def parse(related):
    return re.findall("[A-Z0-9]{10}", related)

def f(df):
    asins = []
    for row in df.iterrows():
        
        asins.append([1])
    
    ret = pd.DataFrame({"asins" : asins})
    return ret
    

subP.map_partitions(f,  meta="str").compute()


In [ ]:
# Q6 new



def g(ser):
    lst = []
    def parse(related):
        if (not pd.isnull(related)):
            lst.extend(re.findall("[A-Z0-9]{10}", related))
    
    ser.apply(parse)
    
    return pd.Series(lst)

def f(df):
    return df.apply(g)

In [ ]:
subR = products[["related"]]

metadata = {"related":"str"}

res = subR.map_partitions(f, meta = metadata)

res = res.rename(columns={"related": "asin"})

res = res.drop_duplicates(split_out=4)#10461397

#res.npartitions

#res.shape[0].compute()

In [ ]:
subA = products[["asin"]]

#subA = subA.set_index("asin")

subA["tempA"] = 1

res2 = res

#res2 = res2.set_index("asin")

res2["tempR"] = 1

jn = subA.merge(res2, how='outer', 
                left_index = True, 
                right_index = True)#on="asin")

In [ ]:
# jn.dask

In [ ]:
(jn.tempA.isna().any() + jn.tempR.isna().any()).compute() >0

In [ ]:
jn.tempA.isna().any().compute()

## FINAL CODE

In [1]:
import dask.dataframe as dd
from dask.distributed import Client
import json
import dask
import numpy as np
import re
import pandas as pd

products = dd.read_csv("products_Release.csv")
reviews = dd.read_csv("user_reviews_Release.csv")

client=Client('172.31.42.114:8786')
client


# Q1
# q1_reviews = reviews.isna().mean().compute()
# q1_products = products.isna().mean().compute()

# Set up for Q1-Q6
product_metadata = {'price':float,  
                    'categories':str, 
                    'asin':str,
                    'related':str}
products = dd.read_csv("products_Release.csv", 
                        usecols=['price',  
                                'categories', 
                                'asin',
                                'related'
                                ],
                       dtype=product_metadata)
reviews_metadata = {'asin':str, 
                    'overall':float}
reviews = dd.read_csv("user_reviews_Release.csv", 
                      usecols=['asin', 'overall'],
                      dtype=reviews_metadata)

bag1 = {}

# Q2
subproducts = products[['asin', 'price']]
subreviews = reviews[['asin', 'overall']]

#optimise with left and not index on products
# joined = subproducts.join(subreviews, 
#                           on="asin", 
#                           how='inner')
joined = dd.merge(subproducts, subreviews, on = 'asin', how = 'inner')

#joined.head()
q2 = joined.corr()['overall'].compute()['price']

# Q3
price_prod = products['price'].describe()
bag1['q3'] = price_prod#.compute()[['mean', 'std', '50%', 'min', 'max']]

# Q4
def get_supercat(x):
    if isinstance(x, float):
        return x
    else:
        return x.strip("[]").split(',')[0]

numCat = products.assign(categories = lambda x: x.categories.apply(
    get_supercat,
    meta=('categories', 'str'))
                        )

numCat = numCat.groupby("categories").count()['asin']
bag1['q4'] = numCat#.compute()[1:]


# Q5
subR = reviews[['asin']]
subP = products[["asin"]]
subP["temp"] = 1

left = dd.merge(subR, subP, how = "left")
bag1['q5'] = left.temp.isna().any()#.compute() + 0


# Q6
def g(ser):
    lst = []
    def parse(related):
        if (not pd.isnull(related)):
            lst.extend(re.findall("[A-Z0-9]{10}", related))

    ser.apply(parse)
    return pd.Series(lst)

def f(df):
    return df.apply(g)

subR = products[["related"]]
metadata = {"related":"str"}
res = subR.map_partitions(f, meta = metadata)
res = res.rename(columns={"related": "asin"})
res = res.drop_duplicates(split_out=4)

subA = products[["asin"]]
subA["tempA"] = 1
res2 = res
res2["tempR"] = 1
jn = subA.merge(res2, how='outer', 
                left_index = True, 
                right_index = True)

q6 = (jn.tempA.isna().any() + jn.tempR.isna().any()).compute() > 0

temp1 = dask.compute(bag1)

temp    
#q1_reviews, q1_products, pre_q2, pre_q3, pre_q4, pre_q5, pre_q6
#q2 = pre_q2['price']
#q3 = pre_q3[['mean', 'std', '50%', 'min', 'max']]
#q4 = pre_q4[1:]
##q5 = pre_q5 + 0
#q6 = pre_q6 > 0

NameError: name 'temp' is not defined